# Image classification and the CIFAR-10 dataset

We will use a standardized dataset called CIFAR-10. CIFAR-10 consists of 60000 images. There are 10 different categories and 6000 images per category. Each image has a size of only 32 by 32 pixels. The small size makes it sometimes difficult for us humans to recognize the correct category, but it simplifies things for our computer model and reduces the computational load required to analyze the images.*
*, by \url{https://www.wolfib.com/Image-Recognition-Intro-Part-1/#prerequisites}

The CIFAR-10 contains 10 classes : plane, car, bird, cat, deer, frog, horse, ship, truck

In [3]:
# sotmax.py
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np 
import tensorflow as tf 
import time
import data_helpers

beginTime = time.time()

# Learning parameters
batch_size = 100
learning_rate = 0.005
max_steps = 1000

# Prepare data : Load the CIFAR-10 dataset
data_sets = data_helpers.load_data()

# NOTE : The dataset contains actually 60000 images but we are spliting them into 2 parts
# - A training set 
# - A testing set (for validating our model)

# According to the same source code used (link Github Below) load_data() fct return the following:
# images_train -> Training set of array 50000*image = 50000*(32.pix*32.pix*3.color_channels(RGB))
# labels_train -> Labels for training set (we're doing supervized learning so all our features are labelized (from 0 to 9 for each class))
# images_test -> 10000 * (3072)
# labels_test -> 10000 images (features) labelized
# classes -> probably a dictionary that associate a for each key(class number) its name as value 
# 

# Enable TensorFlow v.1.x compatibility (version installed latest v.2.x)
tf.compat.v1.disable_eager_execution()

# Define placeholders
images_placeholder = tf.compat.v1.placeholder(tf.float32, shape=[None, 3072])
labels_placeholder = tf.compat.v1.placeholder(tf.int64, shape=[None])

# Define variables
weights = tf.Variable(tf.zeros([3072,10]))
biases = tf.Variable(tf.zeros([10]))

# NOTE : An image is represented by a linear array of 3072 values. 
# Each value is multiplied by a weight parameter and the results are summed up to arrive at a single result - the image's score for a specific class
# Well scores are also called logits that will be converted into probabilities later.

'''
Image = [0.5 0.8 0.1 (3072...) 0.2 0.3]*(3072*10 matrix) + (3072 biases(rows NULL in our case)) -> Result(10 scores)
'''

# Begin feedforward : Apply the multiplication and getting logits
logits = tf.matmul(images_placeholder,weights) + biases

# Define the loss function calculated based on the Cross-Entropy -> well there is an activation before to turn into probabilities
# NOTE: Loss is the difference between the predition and the actual result(the label in this case)
loss = tf.compat.v1.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits,
  labels_placeholder))

# Define the training operation
train_step = tf.compat.v1.train.GradientDescentOptimizer(learning_rate).minimize(loss)

# Operation comparing prediction with true label
correct_prediction = tf.equal(tf.argmax(logits, 1), labels_placeholder)

# Operation calculating the accuracy of our predictions
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# -----------------------------------------------------------------------------
# Run the TensorFlow graph
# -----------------------------------------------------------------------------

# Initialize all Tensor variables
init = tf.compat.v1.global_variables_initializer()

with tf.compat.v1.Session as sess:
    sess.run(init)
    # Repeat for each epochs
    for i in range(max_steps):
        # Generate input data batch
        indices = np.random.choice(data_sets['images_train'].shape[0], batch_size)
        images_batch = data_sets['images_train'][indices]
        labels_batch = data_sets['labels_train'][indices]

        # Periodically print out the model's current accuracy
        if i % 100 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={
                images_placeholder: images_batch, labels_placeholder: labels_batch})
            print('Step {:5d}: training accuracy {:g}'.format(i, train_accuracy))
        # Perform a single training step
        sess.run(train_step, feed_dict={images_placeholder: images_batch,
        labels_placeholder: labels_batch})
    
    # After finishing the training, evaluate on the test set
    test_accuracy = sess.run(accuracy, feed_dict={images_placeholder: data_sets['images_test'],
    labels_placeholder: data_sets['labels_test']})
    print('Test accuracy {:g}'.format(test_accuracy))

endTime = time.time()
print('Total time: {:5.2f}s'.format(endTime - beginTime))





ValueError: `labels.shape.rank` must equal `logits.shape.rank - 1`. Received: labels.shape=(None, 10) of rank 2 and logits.shape=(None, 10) of rank 2

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
import time
import data_helpers

beginTime = time.time()

# Parameter definitions
batch_size = 100
learning_rate = 0.005
max_steps = 1000

# Uncommenting this line removes randomness
# You'll get exactly the same result on each run
# np.random.seed(1)

# Prepare data
data_sets = data_helpers.load_data()

# -----------------------------------------------------------------------------
# Prepare the TensorFlow graph
# (We're only defining the graph here, no actual calculations taking place)
# -----------------------------------------------------------------------------

# Define input placeholders
images_placeholder = tf.placeholder(tf.float32, shape=[None, 3072])
labels_placeholder = tf.placeholder(tf.int64, shape=[None])

# Define variables (these are the values we want to optimize)
weights = tf.Variable(tf.zeros([3072, 10]))
biases = tf.Variable(tf.zeros([10]))

# Define the classifier's result
logits = tf.matmul(images_placeholder, weights) + biases

# Define the loss function
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,
  labels=labels_placeholder))

# Define the training operation
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

# Operation comparing prediction with true label
correct_prediction = tf.equal(tf.argmax(logits, 1), labels_placeholder)

# Operation calculating the accuracy of our predictions
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# -----------------------------------------------------------------------------
# Run the TensorFlow graph
# -----------------------------------------------------------------------------

with tf.Session() as sess:
  # Initialize variables
  sess.run(tf.global_variables_initializer())

  # Repeat max_steps times
  for i in range(max_steps):

    # Generate input data batch
    indices = np.random.choice(data_sets['images_train'].shape[0], batch_size)
    images_batch = data_sets['images_train'][indices]
    labels_batch = data_sets['labels_train'][indices]

    # Periodically print out the model's current accuracy
    if i % 100 == 0:
      train_accuracy = sess.run(accuracy, feed_dict={
        images_placeholder: images_batch, labels_placeholder: labels_batch})
      print('Step {:5d}: training accuracy {:g}'.format(i, train_accuracy))

    # Perform a single training step
    sess.run(train_step, feed_dict={images_placeholder: images_batch,
      labels_placeholder: labels_batch})

  # After finishing the training, evaluate on the test set
  test_accuracy = sess.run(accuracy, feed_dict={
    images_placeholder: data_sets['images_test'],
    labels_placeholder: data_sets['labels_test']})
  print('Test accuracy {:g}'.format(test_accuracy))

endTime = time.time()
print('Total time: {:5.2f}s'.format(endTime - beginTime))

2023-11-24 15:57:26.716407: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-24 15:57:56.621192: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


AttributeError: module 'tensorflow' has no attribute 'placeholder'